In [1]:
import os

import openai
from llama_index.embeddings.openai import OpenAIEmbedding

import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore # pip install llama-index-vector-stores-chroma
from llama_index.core import StorageContext
from llama_index.core import Settings

In [2]:
# https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings/
# https://docs.llamaindex.ai/en/stable/examples/embeddings/OpenAI/

openai.api_key = os.environ["OPENAI_API_KEY"]
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# initialize index
index = None
def initialize_index():
    global index
    collection_name = "MyCollection"
    db = chromadb.PersistentClient(path="./chroma_db")
    chroma_collection = db.get_or_create_collection(collection_name)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    index = VectorStoreIndex.from_vector_store(
        vector_store = vector_store,
        embed_model=embed_model
    )

initialize_index()

In [3]:
# test simple queries of database

query_engine = index.as_query_engine()
response = query_engine.query("What kind of documents are in your database?")
print(response)


The documents in the database contain information about the potential use of the paca as a food supply for people in the tropics and the evolutionary background of the paca species, including their origin in South America and their migration to North America after the Great American Interchange.


In [4]:
# test chat with database

# https://docs.llamaindex.ai/en/stable/examples/chat_engine/chat_engine_openai/

from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")

system_prompt = 'You are a helpful and friendly chatbot.'

chat_engine = index.as_chat_engine(
    chat_mode="openai",
    llm=llm,
    system_prompt = system_prompt,
    verbose=True
)

In [5]:
# non streaming chat
response = chat_engine.chat(
    "Use the tool to answer: what kind of documents are in your database?"
)

Added user message to memory: Use the tool to answer: what kind of documents are in your database?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What kind of documents are in your database?"}
Got output: The documents in the database include information on various topics such as the potential use of animals by humans, evolutionary backgrounds of species, and taxonomic classifications.



In [6]:
print(response.response)

The documents in the database cover a range of topics, including:

- The potential use of animals by humans
- Evolutionary backgrounds of species
- Taxonomic classifications
